In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import random
device = 'cuda' if torch.cuda.is_available() else 'cpu'
import os,sys
opj = os.path.join
from tqdm import tqdm
import acd
from copy import deepcopy
from model_mnist import LeNet5
from visualize import *
import dset_mnist as dset
import foolbox
sys.path.append('../trim')
from transforms_torch import transform_bandpass, tensor_t_augment, batch_fftshift2d, batch_ifftshift2d
from trim import *
from util import *
from attributions import *
from captum.attr import *
import warnings
warnings.filterwarnings("ignore")
# disentangled vae
sys.path.append('../disentangling-vae')
from collections import defaultdict
import vae_trim, vae_trim_viz
from disvae.utils.modelIO import save_model, load_model, load_metadata
from disvae.models.losses import get_loss_f

### Train model

In [ ]:
args = vae_trim.parse_arguments()
args.loss = "btcvae"
args.reg_anneal = 0
args.btcvae_B = 9
args.attr_lamb = 0
name = args.loss + "_B_" + str(args.btcvae_B)
args.name = name

In [ ]:
# train and evaluate model
vae_trim.main(args)

In [ ]:
# generate visualization
args = vae_trim_viz.parse_arguments()
args.name = name
args.plots = "all"
args.n_rows = 10
args.n_cols = 10
vae_trim_viz.main(args)

### Test model

In [ ]:
args = vae_trim.parse_arguments()
args.name = name
# results dir
exp_dir = os.path.join(vae_trim.RES_DIR, args.name)

# load dataloaders
train_loader, test_loader = dset.load_data(args.batch_size, args.eval_batchsize, device)
metadata = load_metadata(exp_dir)

# load model
model = load_model(exp_dir, is_gpu=not args.no_cuda)

In [ ]:
# loss
loss_f = get_loss_f(args.loss,
                    n_data=len(test_loader.dataset),
                    device=device,
                    **vars(args))

# evaluate on testset
storer = defaultdict(list)
for data, _ in tqdm(test_loader, leave=False, disable=args.no_progress_bar):
    data = data.to(device)
    recon_batch, latent_dist, latent_sample = model(data)
    _ = loss_f(data, recon_batch, latent_dist, model.training,
                    storer, latent_sample=latent_sample)    
    losses = {k: sum(v) / len(test_loader) for k, v in storer.items()}
    break
print(losses)